In [1]:
import org.apache.spark.SparkConf
import org.apache.spark.SparkContext
import org.apache.spark.SparkContext._
import org.apache.spark.rdd.RDD
import org.apache.spark.rdd.PairRDDFunctions._
import org.apache.spark.SparkContext
import org.apache.spark.graphx.Edge
import org.apache.spark.graphx.TripletFields
import org.apache.spark.graphx.Graph
import org.apache.spark.graphx.VertexId
import org.apache.spark.graphx.EdgeTriplet
import org.apache.spark.graphx.GraphLoader
import org.apache.spark._
import org.apache.spark.graphx._
import scala.util.Random
import scala.io.Source
import java.io._

Intitializing Scala interpreter ...

Spark Web UI available at http://c7bc9d3779d0:4040
SparkContext available as 'sc' (version = 2.3.1, master = local[*], app id = local-1618522443627)
SparkSession available as 'spark'


2021-04-15 21:34:02 WARN  NativeCodeLoader:62 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


import org.apache.spark.SparkConf
import org.apache.spark.SparkContext
import org.apache.spark.SparkContext._
import org.apache.spark.rdd.RDD
import org.apache.spark.rdd.PairRDDFunctions._
import org.apache.spark.SparkContext
import org.apache.spark.graphx.Edge
import org.apache.spark.graphx.TripletFields
import org.apache.spark.graphx.Graph
import org.apache.spark.graphx.VertexId
import org.apache.spark.graphx.EdgeTriplet
import org.apache.spark.graphx.GraphLoader
import org.apache.spark._
import org.apache.spark.graphx._
import scala.util.Random
import scala.io.Source
import java.io._


In [2]:
class VertexAttr(
    var degree : Int = 0,
    var initialLabel : Boolean = false,
    var vertexLabels : Array[Float] = Array[Float](0, 0, 0)
) extends java.io.Serializable {     
    
    def setDegree(degree: Int) = {
        this.degree = degree
    }
}

defined class VertexAttr


In [3]:
def readLabel(filename: String) = {
    val pairs = 
        for {
            line <- Source.fromFile(filename).getLines()
            split = line.split(" ").map(_.trim)
            name = split.head.toInt
            scores = split.tail(0).toInt
        } yield (name -> scores)
        pairs.toMap
}

readLabel: (filename: String)scala.collection.immutable.Map[Int,Int]


In [4]:
// Load initial Graph
val initialGraph = GraphLoader.edgeListFile(sc, "./data/com-dblp/com-dblp-ungraph-custom.txt", canonicalOrientation=false)
val initialLabel = readLabel("./data/com-dblp/com-dblp-label-custom.txt")
val n_label = initialLabel.map({ case (k, v) => v }).max + 1

initialGraph: org.apache.spark.graphx.Graph[Int,Int] = org.apache.spark.graphx.impl.GraphImpl@24d0133
initialLabel: scala.collection.immutable.Map[Int,Int] = Map(341262 -> 13164, 140385 -> 13164, 238478 -> 13428, 255433 -> 13428, 143890 -> 13428, 231297 -> 13428, 31885 -> 13164, 321404 -> 13164, 338021 -> 13164, 164871 -> 13428, 80586 -> 13428, 182424 -> 13428, 259620 -> 13428, 251598 -> 13428, 332709 -> 13428, 168780 -> 13428, 319598 -> 13428, 266519 -> 13428, 204758 -> 13428, 215513 -> 13164, 212578 -> 13164, 190394 -> 13428, 256215 -> 13428, 131212 -> 13164, 139973 -> 13164, 230575 -> 13428, 186425 -> 13428, 250357 -> 13164, 388818 -> 13428, 137573 -> 13428, 325737 -> 13164, 217990 -> 13164, 261943 -> 13164, 128542 -> 13164, 278252 -> 13164, 295703 -> 13164, 107274 -> 13164, 161548 -...

In [5]:
val rand = new Random(42)
val probability = 0.5

def random_label(random: Random): Int = {
    random.nextInt(3)
}

def transform(VD: VertexId): VertexAttr = {
    var labelArray = Array.ofDim[Float](n_label)
    var va = new VertexAttr(0, false, labelArray);
    
    if (initialLabel.contains(VD.toInt)) {
        var label = initialLabel(VD.toInt)
        
        if (rand.nextFloat() < probability){
            labelArray(label) = 1
            va.initialLabel = true
            va.vertexLabels = labelArray
        }
    }
    
    va;
}


rand: scala.util.Random = scala.util.Random@69ef94d7
probability: Double = 0.5
random_label: (random: scala.util.Random)Int
transform: (VD: org.apache.spark.graphx.VertexId)VertexAttr


In [6]:
// Transform vertices into VertexAttr with a random label assigned
val graph = initialGraph.mapVertices({ case (id, attr) => transform(id)})

graph: org.apache.spark.graphx.Graph[VertexAttr,Int] = org.apache.spark.graphx.impl.GraphImpl@67798500


In [7]:
// Populate Graph degree for each vertices and create final graph
val vertex_rdd = graph.degrees.zip(graph.vertices).map({ case (vDeg, vAttr) => vAttr._2.setDegree(vDeg._2) ; vAttr})
val edge_rdd = graph.edges
val graph2 = Graph(vertex_rdd, edge_rdd)

// Edges go both sides ways
val finalGraph = Graph(graph2.vertices, graph2.edges.union(graph2.edges.reverse))

vertex_rdd: org.apache.spark.rdd.RDD[(org.apache.spark.graphx.VertexId, VertexAttr)] = MapPartitionsRDD[20] at map at <console>:66
edge_rdd: org.apache.spark.graphx.EdgeRDD[Int] = EdgeRDDImpl[14] at RDD at EdgeRDD.scala:41
graph2: org.apache.spark.graphx.Graph[VertexAttr,Int] = org.apache.spark.graphx.impl.GraphImpl@2162bcdf
finalGraph: org.apache.spark.graphx.Graph[VertexAttr,Int] = org.apache.spark.graphx.impl.GraphImpl@54bbb4d


In [8]:
// Vertex Progam
def VertexProgram(id : VertexId, 
                  VD : VertexAttr, 
                  A : Array[Float]) : VertexAttr  = {
    var newVertexAttr = new VertexAttr()
    newVertexAttr.degree = VD.degree
    newVertexAttr.initialLabel = VD.initialLabel
    newVertexAttr.vertexLabels = VD.vertexLabels
    
    if(VD.initialLabel == false){
        newVertexAttr.vertexLabels = A.map(_ / VD.degree)
    }
    newVertexAttr
}

// Send Message
def SendMsg(triplet : EdgeTriplet[VertexAttr, Int]): Iterator[(VertexId, Array[Float])] = {
    Iterator((triplet.dstId, triplet.srcAttr.vertexLabels))
}

// Merge Message
def MergeMsg(A : Array[Float],
             B : Array[Float]) : Array[Float] = {
    val mergeMsg = new Array[Float](A.length);
    for ( i <- 0 to (A.length - 1)) {
        mergeMsg(i) = A(i) + B(i);
    }
    mergeMsg;
}

VertexProgram: (id: org.apache.spark.graphx.VertexId, VD: VertexAttr, A: Array[Float])VertexAttr
SendMsg: (triplet: org.apache.spark.graphx.EdgeTriplet[VertexAttr,Int])Iterator[(org.apache.spark.graphx.VertexId, Array[Float])]
MergeMsg: (A: Array[Float], B: Array[Float])Array[Float]


In [9]:
// finalGraph.vertices.foreach(v => println(v._1, v._2.degree, v._2.initialLabel, v._2.vertexLabels.mkString(" ")))

In [ ]:
val initialMsg = Array.ofDim[Float](n_label)
val maxIter = 2

val graphLP = finalGraph.pregel(initialMsg, maxIter, EdgeDirection.Either)(
    (id, VD, newLabelsArray) => VertexProgram(id, VD, newLabelsArray),
    tr => SendMsg(tr),
    (a, b) => MergeMsg(a, b)
)

2021-04-15 21:34:22 WARN  BlockManager:66 - Block rdd_11_0 could not be removed as it was not found on disk or in memory
2021-04-15 21:34:22 ERROR Executor:91 - Exception in task 0.0 in stage 4.0 (TID 10)
java.lang.OutOfMemoryError: Java heap space
	at scala.reflect.ManifestFactory$$anon$11.newArray(Manifest.scala:133)
	at scala.reflect.ManifestFactory$$anon$11.newArray(Manifest.scala:131)
	at scala.Array$.ofDim(Array.scala:218)
	at $line10.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.transform(<console>:65)
	at $line11.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$anonfun$1.apply(<console>:64)
	at $line11.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$anonfun$1.apply(<console>:64)
	at org.apache.spark.graphx.impl.VertexPartitionBaseOps.map(VertexPartitionBaseOps.scala:61)
	at org.apache.spark.graphx.impl.GraphImpl$$anonfun$mapVertices$1.apply(GraphImpl.scala:136)
	at org.apache.spark.grap

2021-04-15 21:34:25 WARN  TaskSetManager:66 - Lost task 0.0 in stage 4.0 (TID 10, localhost, executor driver): java.lang.OutOfMemoryError: Java heap space
	at scala.reflect.ManifestFactory$$anon$11.newArray(Manifest.scala:133)
	at scala.reflect.ManifestFactory$$anon$11.newArray(Manifest.scala:131)
	at scala.Array$.ofDim(Array.scala:218)
	at $line10.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.transform(<console>:65)
	at $line11.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$anonfun$1.apply(<console>:64)
	at $line11.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$anonfun$1.apply(<console>:64)
	at org.apache.spark.graphx.impl.VertexPartitionBaseOps.map(VertexPartitionBaseOps.scala:61)
	at org.apache.spark.graphx.impl.GraphImpl$$anonfun$mapVertices$1.apply(GraphImpl.scala:136)
	at org.apache.spark.graphx.impl.GraphImpl$$anonfun$mapVertices$1.apply(GraphImpl.scala:136)
	at scala.collection.Itera

In [ ]:
graphLP.vertices.foreach(v => println(v._1, v._2.initialLabel, v._2.vertexLabels.mkString(",")))